In [ ]:
#
# Program 7.1: Electric field hockey (hockey.ipynb)
# J Wang, Computational modeling and visualization with Python
#

import vpython as vp, numpy as np, ode
import sys
vec = vp.vector

def hockey(Y, t):               # return eqn of motion
    accel = 0.
    for i in range(len(loc)):
        r = Y[0]-loc[i][0:2]
        s=np.sqrt(r[0]**2+r[1]**2)
        accel += Q[i]*r/s**3
    return [Y[1], q*accel]      # list for non-vectorized solver

a, b, w = 1., 0.5, 0.125                  # rink size, goal width
q, qcor, qmid, qcen = -1.0, 1.0, -2., 2.  # Qs: puck, cor., mid, cen.
Q = [qcor, qmid, qcor, qcor, qmid, qcor, qcen]  # charges, locations
loc = [[-a, b, 0], [0, b, 0], [a, b, 0], [a,-b, 0], [0,-b, 0], [-a,-b, 0], [0,0, 0]]

scene = vp.canvas(title='Electric hockey', background=vec(.2,.5,1))
puck  = vp.sphere(pos=vec(-a,0,0), radius = 0.05, make_trail=True) # trail 
rink  = vp.curve(pos=loc[:-1]+[loc[0]], radius=0.01)    # closed curve
goalL = vp.curve(pos=[(-a,w,0),(-a,-w,0)], color=vec(0,1,0), radius=.02)
goalR = vp.curve(pos=[( a,w,0),( a,-w,0)], color=vec(0,1,0), radius=.02)
for i in range(len(loc)):       # charges, red if Q>0, blue if Q<0
    color = vec(1,0,0) if Q[i]>0 else vec(0,0,1)  
    vp.sphere(pos=vec(loc[i][0],loc[i][1],0), radius = 0.05, color=color)

if (sys.version_info[0] < 3):
    v, theta = input('enter speed, theta; eg, 2.2, 19:')   # try 2.2, 18.5
else:
    v, theta = eval(input('enter speed, theta; eg, 2.2, 19:'))
v, theta = min(4, v), max(1,theta)*np.pi/180.      # check valid input
Y = np.array([[-a,0], [v*np.cos(theta), v*np.sin(theta)]])
while True:
    vp.rate(200)
    Y = ode.RK45n(hockey, Y, t=0., h=0.002)
    x, y = Y[0][0], Y[0][1]
    if (abs(x) > a or abs(y) >b): 
        txt = 'Goal!' if (x > 0 and abs(y) < w) else 'Miss!'
        vp.label(pos=vec(x, y+.2,0), text=txt, box=False)
        break
    puck.pos = vec(x,y,0)
